In [5]:
import pandas as pd

# Load the raw datasets
certificates = pd.read_csv('../data/certificates.csv')
recommendations = pd.read_csv('../data/recommendations.csv')

# Aggregate recommendations
reco_agg = recommendations.groupby("LMK_KEY").agg({
    "IMPROVEMENT_ID": "nunique",
    "IMPROVEMENT_SUMMARY_TEXT": lambda x: ', '.join(set(x.dropna()))
}).reset_index().rename(columns={
    "IMPROVEMENT_ID": "N_RECOMMENDATIONS",
    "IMPROVEMENT_SUMMARY_TEXT": "ALL_RECOMMENDATIONS"
})

# Merge datasets
df_merged = pd.merge(certificates, reco_agg, on="LMK_KEY", how="left")

# Encode energy ratings
rating_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
df_merged['CURRENT_ENERGY_RATING'] = df_merged['CURRENT_ENERGY_RATING'].map(rating_map)
df_merged['POTENTIAL_ENERGY_RATING'] = df_merged['POTENTIAL_ENERGY_RATING'].map(rating_map)

# Derived features
df_merged['TOTAL_COST_CURRENT'] = df_merged[['LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT', 'HOT_WATER_COST_CURRENT']].sum(axis=1)
df_merged['COST_PER_AREA'] = df_merged['TOTAL_COST_CURRENT'] / df_merged['TOTAL_FLOOR_AREA']
df_merged['CO2_PER_AREA'] = df_merged['CO2_EMISSIONS_CURRENT'] / df_merged['TOTAL_FLOOR_AREA']

# Create full address
df_merged['PROPERTY_ADDRESS'] = df_merged[['ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE']]\
    .fillna('')\
    .agg(', '.join, axis=1)\
    .str.replace(', ,', ',', regex=False)\
    .str.replace(', ,', ',', regex=False)\
    .str.strip(', ')

# Drop missing values
critical_cols = [
    'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_RATING',
    'TOTAL_COST_CURRENT', 'COST_PER_AREA', 'CO2_PER_AREA',
    'LMK_KEY', 'PROPERTY_ADDRESS'
]
df_merged.dropna(subset=critical_cols, inplace=True)

# Keep only final 15 features + ID/address fields
final_feature_cols = [
    'CURRENT_ENERGY_RATING', 'CO2_PER_AREA', 'TOTAL_FLOOR_AREA',
    'MULTI_GLAZE_PROPORTION', 'EXTENSION_COUNT', 'NUMBER_HABITABLE_ROOMS',
    'LOW_ENERGY_LIGHTING', 'NUMBER_OPEN_FIREPLACES', 'FLOOR_HEIGHT',
    'PHOTO_SUPPLY', 'FIXED_LIGHTING_OUTLETS_COUNT', 'REPORT_TYPE',
    'N_RECOMMENDATIONS', 'TOTAL_COST_CURRENT', 'COST_PER_AREA'
]

# Build final lookup_df
lookup_df = df_merged[['LMK_KEY', 'PROPERTY_ADDRESS'] + final_feature_cols]
lookup_df['ADDRESS_LOWER'] = lookup_df['PROPERTY_ADDRESS'].str.lower()
lookup_df.set_index('LMK_KEY', inplace=True)

# Save
lookup_df.to_csv('../models/epc_feature_lookup.csv')
print("lookup_df generated and saved to '../models/epc_feature_lookup.csv'")

/tmp/ipykernel_35255/2267696774.py:4: DtypeWarning: Columns (15,37,83) have mixed types. Specify dtype option on import or set low_memory=False.
  certificates = pd.read_csv('../data/certificates.csv')
/tmp/ipykernel_35255/2267696774.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_df['ADDRESS_LOWER'] = lookup_df['PROPERTY_ADDRESS'].str.lower()


lookup_df generated and saved to '../models/epc_feature_lookup.csv'
